<a href="https://colab.research.google.com/github/woneuy01/AudioToText/blob/master/Reddit_sample2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import requests
import json
import csv
import time
import datetime

In [0]:
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=100&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [0]:
def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
    
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair))
    subStats[sub_id] = subData

In [0]:
#Subreddit to query
sub='coronavirus'
#before and after dates
before = "1588377600" #June 2
after = "1588291200"  #June 1st 
query = ""
subCount = 0
subStats = {}

In [9]:
data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    try:
      data = getPushshiftData(query, after, before, sub)
    except ValueError:
      print("response content is not valid json")
    
print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1588291200&before=1588377600&subreddit=coronavirus
100
2020-05-01 01:27:42
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1588296462&before=1588377600&subreddit=coronavirus
100
2020-05-01 03:39:16
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1588304356&before=1588377600&subreddit=coronavirus
100
2020-05-01 06:30:17
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1588314617&before=1588377600&subreddit=coronavirus
100
2020-05-01 09:54:40
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1588326880&before=1588377600&subreddit=coronavirus
100
2020-05-01 11:51:22
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1588333882&before=1588377600&subreddit=coronavirus
100
2020-05-01 13:27:24
https://api.pushshift.io/reddit/search/submission/?title=&size=100&after=1588339644&before=1588377600&subreddit=

In [11]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

1321 submissions have added to list
1st entry is:
Coronavirus Australia: China lashes out at Scott Morrison as diplomatic tensions escalate created: 2020-05-01 00:01:55
Last entry is:
Brazil’s Bolsonaro sits on a ticking coronavirus time bomb created: 2020-05-01 23:54:39


In [0]:
# def updateSubs_file():
#     upload_count = 0
#     location = ""
#     print("input filename of submission file, please add .csv")
    #filename = input()
    #file = location + filename
    # with open(file, 'w', newline='', encoding='utf-8') as file: 
    #     a = csv.writer(file, delimiter=',')
    #     headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
    #     a.writerow(headers)
    #     for sub in subStats:
    #         a.writerow(subStats[sub][0])
    #         upload_count+=1
               
        #print(str(upload_count) + " submissions have been uploaded")


In [75]:
posts=[]
for sub in subStats:
  posts.append(subStats[sub][0]) 
 
df=pd.DataFrame(posts, columns=["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"])
df.head()

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
0,gb7i90,Coronavirus Australia: China lashes out at Sco...,https://www.news.com.au/world/coronavirus/aust...,Scbadiver,1,2020-05-01 00:01:55,7,/r/Coronavirus/comments/gb7i90/coronavirus_aus...,NaN
1,gb7irp,"Delta, Frontier &amp; American Airlines All Jo...",https://www.doctorofcredit.com/delta-frontier-...,white_men_disease,1,2020-05-01 00:02:47,0,/r/Coronavirus/comments/gb7irp/delta_frontier_...,USA
2,gb7jqn,Texas Walmart employees share nearly $18.9 mil...,https://www.kwtx.com/content/news/Texas-Walmar...,BoyYeetzWorld,1,2020-05-01 00:04:24,25,/r/Coronavirus/comments/gb7jqn/texas_walmart_e...,USA
3,gb7k7t,The International Dota 2 Championships (bigges...,http://blog.dota2.com/2020/04/the-internationa...,pinoygs,1,2020-05-01 00:05:07,9,/r/Coronavirus/comments/gb7k7t/the_internation...,World
4,gb7mc3,China exports billions of masks from March to ...,http://www.chinadaily.com.cn/a/202004/26/WS5ea...,flying-fat-cat,1,2020-05-01 00:08:33,25,/r/Coronavirus/comments/gb7mc3/china_exports_b...,Central &amp; East Asia


In [0]:
#df["Title"]=df["Title"].str.decode('utf-8')

In [76]:
df.sort_values("Total No. of Comments", axis = 0, ascending = False, 
                 inplace = True, na_position ='last') 
df.head()

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
809,gbldq9,Coronavirus homeschooling: 77 percent of paren...,https://www.foxnews.com/lifestyle/coronavirus-...,Sumit316,1,2020-05-01 16:33:11,2184,/r/Coronavirus/comments/gbldq9/coronavirus_hom...,USA
627,gbih0c,"Daily Discussion Post - May 01 | Questions, im...",https://www.reddit.com/r/Coronavirus/comments/...,AutoModerator,1,2020-05-01 13:52:38,2112,/r/Coronavirus/comments/gbih0c/daily_discussio...,NaN
656,gbiwcm,Chinese state media releases animated propagan...,https://www.abc.net.au/news/2020-05-01/china-s...,Classic-Durian,1,2020-05-01 14:17:32,1866,/r/Coronavirus/comments/gbiwcm/chinese_state_m...,NaN
189,gbaq2w,Japanese island suffering second wave of coron...,https://www.telegraph.co.uk/news/2020/04/29/ja...,N19864,1,2020-05-01 03:31:26,1737,/r/Coronavirus/comments/gbaq2w/japanese_island...,Central &amp; East Asia
592,gbhz0c,After a day of armed protesters and a GOP laws...,https://www.washingtonpost.com/nation/2020/05/...,HeinieKaboobler,1,2020-05-01 13:20:27,1472,/r/Coronavirus/comments/gbhz0c/after_a_day_of_...,USA


In [78]:
df=df[df["Total No. of Comments"]>300]
df.head()

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
809,gbldq9,Coronavirus homeschooling: 77 percent of paren...,https://www.foxnews.com/lifestyle/coronavirus-...,Sumit316,1,2020-05-01 16:33:11,2184,/r/Coronavirus/comments/gbldq9/coronavirus_hom...,USA
627,gbih0c,"Daily Discussion Post - May 01 | Questions, im...",https://www.reddit.com/r/Coronavirus/comments/...,AutoModerator,1,2020-05-01 13:52:38,2112,/r/Coronavirus/comments/gbih0c/daily_discussio...,NaN
656,gbiwcm,Chinese state media releases animated propagan...,https://www.abc.net.au/news/2020-05-01/china-s...,Classic-Durian,1,2020-05-01 14:17:32,1866,/r/Coronavirus/comments/gbiwcm/chinese_state_m...,NaN
189,gbaq2w,Japanese island suffering second wave of coron...,https://www.telegraph.co.uk/news/2020/04/29/ja...,N19864,1,2020-05-01 03:31:26,1737,/r/Coronavirus/comments/gbaq2w/japanese_island...,Central &amp; East Asia
592,gbhz0c,After a day of armed protesters and a GOP laws...,https://www.washingtonpost.com/nation/2020/05/...,HeinieKaboobler,1,2020-05-01 13:20:27,1472,/r/Coronavirus/comments/gbhz0c/after_a_day_of_...,USA


In [0]:
#produce excel
#caution encoding="uft-8" does not work. only "utf-8-sig"
#df.to_csv(str(after)+".csv",encoding="utf-8-sig")

In [0]:
#df.title text file no index&header
df["Title"].to_csv(str(after)+".txt",index=False,header=None, encoding="utf-8-sig")